# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 8:44AM,241269,10,0085840631,ISDIN Corporation,Released
1,Jul 7 2022 8:44AM,241269,10,0085840638,ISDIN Corporation,Released
2,Jul 7 2022 8:44AM,241269,10,0085840634,ISDIN Corporation,Released
3,Jul 7 2022 8:44AM,241269,10,0085840660,ISDIN Corporation,Released
4,Jul 7 2022 8:44AM,241269,10,0085840663,ISDIN Corporation,Released
5,Jul 7 2022 8:44AM,241269,10,0085840666,ISDIN Corporation,Released
6,Jul 7 2022 8:44AM,241269,10,0085840694,ISDIN Corporation,Released
7,Jul 7 2022 8:44AM,241269,10,0085840690,ISDIN Corporation,Released
8,Jul 7 2022 8:44AM,241269,10,0085840689,ISDIN Corporation,Released
9,Jul 7 2022 8:44AM,241269,10,0085840731,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
54,241264,Released,1
55,241265,Released,38
56,241266,Released,38
57,241267,Released,2
58,241269,Released,34


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241264,NaN,NaN,1.0
241265,NaN,NaN,38.0
241266,NaN,NaN,38.0
241267,NaN,NaN,2.0
241269,NaN,NaN,34.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241163,0.0,0.0,58.0
241164,0.0,0.0,8.0
241169,2.0,2.0,28.0
241174,0.0,1.0,0.0
241178,0.0,2.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241163,0,0,58
241164,0,0,8
241169,2,2,28
241174,0,1,0
241178,0,2,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241163,0,0,58
1,241164,0,0,8
2,241169,2,2,28
3,241174,0,1,0
4,241178,0,2,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241163,,,58
1,241164,,,8
2,241169,2,2,28
3,241174,,1,
4,241178,,2,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 8:44AM,241269,10,ISDIN Corporation
34,Jul 7 2022 8:41AM,241266,10,ISDIN Corporation
72,Jul 7 2022 8:41AM,241265,10,ISDIN Corporation
110,Jul 7 2022 8:38AM,241267,10,ISDIN Corporation
112,Jul 7 2022 8:36AM,241262,10,ISDIN Corporation
160,Jul 7 2022 8:33AM,241264,10,Bio-PRF
161,Jul 7 2022 8:30AM,241250,10,Eywa Pharma Inc.
162,Jul 7 2022 8:30AM,241251,10,"Citieffe, Inc."
163,Jul 7 2022 8:30AM,241242,10,ISDIN Corporation
173,Jul 7 2022 8:30AM,241243,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 7 2022 8:44AM,241269,10,ISDIN Corporation,,,34
1,Jul 7 2022 8:41AM,241266,10,ISDIN Corporation,,,38
2,Jul 7 2022 8:41AM,241265,10,ISDIN Corporation,,,38
3,Jul 7 2022 8:38AM,241267,10,ISDIN Corporation,,,2
4,Jul 7 2022 8:36AM,241262,10,ISDIN Corporation,,,48
5,Jul 7 2022 8:33AM,241264,10,Bio-PRF,,,1
6,Jul 7 2022 8:30AM,241250,10,Eywa Pharma Inc.,,,1
7,Jul 7 2022 8:30AM,241251,10,"Citieffe, Inc.",,,1
8,Jul 7 2022 8:30AM,241242,10,ISDIN Corporation,,,32
9,Jul 7 2022 8:30AM,241243,10,ISDIN Corporation,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 8:44AM,241269,10,ISDIN Corporation,34,,
1,Jul 7 2022 8:41AM,241266,10,ISDIN Corporation,38,,
2,Jul 7 2022 8:41AM,241265,10,ISDIN Corporation,38,,
3,Jul 7 2022 8:38AM,241267,10,ISDIN Corporation,2,,
4,Jul 7 2022 8:36AM,241262,10,ISDIN Corporation,48,,
5,Jul 7 2022 8:33AM,241264,10,Bio-PRF,1,,
6,Jul 7 2022 8:30AM,241250,10,Eywa Pharma Inc.,1,,
7,Jul 7 2022 8:30AM,241251,10,"Citieffe, Inc.",1,,
8,Jul 7 2022 8:30AM,241242,10,ISDIN Corporation,32,,
9,Jul 7 2022 8:30AM,241243,10,ISDIN Corporation,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 8:44AM,241269,10,ISDIN Corporation,34,,
1,Jul 7 2022 8:41AM,241266,10,ISDIN Corporation,38,,
2,Jul 7 2022 8:41AM,241265,10,ISDIN Corporation,38,,
3,Jul 7 2022 8:38AM,241267,10,ISDIN Corporation,2,,
4,Jul 7 2022 8:36AM,241262,10,ISDIN Corporation,48,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 8:44AM,241269,10,ISDIN Corporation,34.0,NaN,NaN
1,Jul 7 2022 8:41AM,241266,10,ISDIN Corporation,38.0,NaN,NaN
2,Jul 7 2022 8:41AM,241265,10,ISDIN Corporation,38.0,NaN,NaN
3,Jul 7 2022 8:38AM,241267,10,ISDIN Corporation,2.0,NaN,NaN
4,Jul 7 2022 8:36AM,241262,10,ISDIN Corporation,48.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 8:44AM,241269,10,ISDIN Corporation,34.0,0.0,0.0
1,Jul 7 2022 8:41AM,241266,10,ISDIN Corporation,38.0,0.0,0.0
2,Jul 7 2022 8:41AM,241265,10,ISDIN Corporation,38.0,0.0,0.0
3,Jul 7 2022 8:38AM,241267,10,ISDIN Corporation,2.0,0.0,0.0
4,Jul 7 2022 8:36AM,241262,10,ISDIN Corporation,48.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,8201575,605.0,1.0,0.0
12,482386,2.0,2.0,0.0
16,723709,1.0,2.0,0.0
19,964919,66.0,1.0,8.0
20,723587,57.0,2.0,2.0
21,1929920,7.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,8201575,605.0,1.0,0.0
1,12,482386,2.0,2.0,0.0
2,16,723709,1.0,2.0,0.0
3,19,964919,66.0,1.0,8.0
4,20,723587,57.0,2.0,2.0
5,21,1929920,7.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,605.0,1.0,0.0
1,12,2.0,2.0,0.0
2,16,1.0,2.0,0.0
3,19,66.0,1.0,8.0
4,20,57.0,2.0,2.0
5,21,7.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,605.0
1,12,Released,2.0
2,16,Released,1.0
3,19,Released,66.0
4,20,Released,57.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,8.0,2.0,0.0
Executing,1.0,2.0,2.0,1.0,2.0,1.0
Released,605.0,2.0,1.0,66.0,57.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,8.0,2.0,0.0
1,Executing,1.0,2.0,2.0,1.0,2.0,1.0
2,Released,605.0,2.0,1.0,66.0,57.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,8.0,2.0,0.0
1,Executing,1.0,2.0,2.0,1.0,2.0,1.0
2,Released,605.0,2.0,1.0,66.0,57.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()